In [1]:
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
import pandas as pd # remove when finished
from implementations import *

In [3]:
# from prepare import *
# from costs import *
# from grid_search import *
# from gradient_descent import *
# from stochastic_gradient_descent import *
# from build_polynomial import *
# from least_squares import *
# from split_data import *
# from ridge_regression import * 
# from proj1_helpers import *

In [4]:
# label, features, ids = load_csv_data(data_path, sub_sample=True)

In [2]:
data_tr = pd.read_csv('data/train.csv')
data_te = pd.read_csv('data/test.csv')
idx_tr = data_tr.Id
idx_te = data_te.Id
# data_tr.head()

In [3]:
features_names = data_tr.columns.drop(['Id','Prediction'])
# features_names

In [6]:
def hist_for_feature(data,dataCol):
    data1 = data.loc[(data[dataCol]!=-999)].copy()
    data1_s = data1.loc[(data.Prediction=='s')].copy()
    data1_b = data1.loc[(data.Prediction=='b')].copy()

    fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(20, 8))
    hist_range = (min(data1_s[dataCol]),max(data1_s[dataCol]))
    ax1.hist(data1_s[dataCol],log=False,range = hist_range,color = '#e0f3db')
    ax1.set_title(dataCol+'_s')
    ax2.hist(data1_b[dataCol],log=False,range = hist_range, color = '#999999')
    ax2.set_title(dataCol+'_b')
    plt.savefig('image/hist_notlog/'+dataCol+'.jpg')
    del ax1,ax2
    plt.close('all')

In [8]:
# for feature in features:
#     hist_for_feature(data_tr,feature)

In [4]:
def missing_rate(data):
    features = data.columns.drop(['Id','Prediction'])
    df_ = pd.DataFrame({'idx':[np.nan]})
    for feature in features:
        data2 = data.loc[(data[feature]==-999)].copy()
        missing_rate = data2[feature].size/data[feature].size
        df1 = pd.DataFrame({feature:[missing_rate]})
        df_ = pd.concat([df_,df1],axis=1)
    df_ = df_.drop(columns='idx')
    df_ = df_.T.rename(columns={0:'rate'})
    return df_

In [5]:
def drop_columns(df_miss):
    names_70 = list(df_miss.loc[df_miss.rate==0.709828].index)
    names_30 = list(df_miss.loc[df_miss.rate==0.399652].index)
    names_phi = list(['PRI_jet_leading_phi','PRI_jet_subleading_phi','PRI_lep_phi','PRI_met_phi','PRI_tau_phi','PRI_jet_num'])
    df_drop_name = {'names_70':names_70,'names_30':names_30,'names_phi':names_phi}
    return df_drop_name

In [6]:
df_miss_tr = missing_rate(data_tr)
df_miss_tr

,rate
DER_mass_MMC,0.152456
DER_mass_transverse_met_lep,0.000000
DER_mass_vis,0.000000
DER_pt_h,0.000000
DER_deltaeta_jet_jet,0.709828
DER_mass_jet_jet,0.709828
DER_prodeta_jet_jet,0.709828
DER_deltar_tau_lep,0.000000
DER_pt_tot,0.000000
DER_sum_pt,0.000000


In [13]:
# replacing the -999 with mode in signal or background

DER_mode_s = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].mean(axis=0)#value_counts().index[0]
DER_mode_b = data_tr.loc[(data_tr['DER_mass_MMC']!=-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].mean(axis=0)#value_counts().index[0]

idx_s = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='s']['DER_mass_MMC'].index
idx_b = data_tr.loc[(data_tr['DER_mass_MMC']==-999)].loc[(data_tr['Prediction'])=='b']['DER_mass_MMC'].index

data_tr.loc[idx_s,'DER_mass_MMC']=DER_mode_s
data_tr.loc[idx_b,'DER_mass_MMC']=DER_mode_b


In [7]:
drop_col = drop_columns(df_miss_tr)
drop_col

{'names_70': ['DER_deltaeta_jet_jet',
  'DER_mass_jet_jet',
  'DER_prodeta_jet_jet',
  'DER_lep_eta_centrality',
  'PRI_jet_subleading_pt',
  'PRI_jet_subleading_eta',
  'PRI_jet_subleading_phi'],
 'names_30': ['PRI_jet_leading_pt',
  'PRI_jet_leading_eta',
  'PRI_jet_leading_phi'],
 'names_phi': ['PRI_jet_leading_phi',
  'PRI_jet_subleading_phi',
  'PRI_lep_phi',
  'PRI_met_phi',
  'PRI_tau_phi',
  'PRI_jet_num']}

'DER_deltaeta_jet_jet','DER_mass_jet_jet','DER_prodeta_jet_jet','DER_lep_eta_centrality','PRI_jet_subleading_pt','PRI_jet_subleading_eta','PRI_jet_subleading_phi': missing rate = 70.9% (PRI_jet_num <= 1)

'PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi': missing rate = 39.9% (PRI_jet_num = 0)

In [8]:
def group_features_by_jet(df):
    return {  
        0: df.loc[df['PRI_jet_num'] == 0].copy(),
        1: df.loc[df['PRI_jet_num'] == 1].copy(),
        2: df.loc[(df['PRI_jet_num'] == 2) | (df['PRI_jet_num'] == 3)].copy(),
    }

In [9]:
DER_mode_te = data_te.loc[(data_te['DER_mass_MMC']!=-999)]['DER_mass_MMC'].mean(axis=0)#value_counts().index[0]
idx_te = data_te.loc[(data_te['DER_mass_MMC']==-999)]['DER_mass_MMC'].index
data_te.loc[idx_te,'DER_mass_MMC']=DER_mode_te

In [10]:
feature_tr_grp = group_features_by_jet(data_tr)
feature_te_grp = group_features_by_jet(data_te)

In [11]:
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30'])))
feature_tr_grp[0] = feature_tr_grp[0].drop(columns = 'PRI_jet_all_pt')# all 0s
feature_tr_grp[1] = feature_tr_grp[1].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70'])))
feature_tr_grp[2] = feature_tr_grp[2].drop(columns = list(set(drop_col['names_phi'])))

feature_te_grp[0] = feature_te_grp[0].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70']+drop_col['names_30'])))
feature_te_grp[0] = feature_te_grp[0].drop(columns = 'PRI_jet_all_pt')# all 0s
feature_te_grp[1] = feature_te_grp[1].drop(columns = list(set(drop_col['names_phi']+drop_col['names_70'])))
feature_te_grp[2] = feature_te_grp[2].drop(columns = list(set(drop_col['names_phi'])))

In [12]:
feature_tr_grp[0]

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,DER_pt_ratio_lep_tau,DER_met_phi_centrality,PRI_tau_pt,PRI_tau_eta,PRI_lep_pt,PRI_lep_eta,PRI_met,PRI_met_sumet
3,100003,b,143.905,81.417,80.943,0.414,3.310,0.414,75.968,2.354,-1.285,22.647,-1.655,53.321,-0.522,31.082,86.062
4,100004,b,175.864,16.915,134.805,16.405,3.891,16.405,57.983,1.056,-1.385,28.209,-2.197,29.774,0.798,2.723,53.131
8,100008,b,105.594,50.559,100.989,4.288,2.904,4.288,65.333,0.675,-1.366,39.008,2.433,26.325,0.210,37.791,129.804
10,100010,b,-999.000,86.240,79.692,27.201,2.338,27.201,81.734,1.750,-1.412,29.718,-0.866,52.016,0.126,51.276,250.178
13,100013,b,82.488,31.663,64.128,8.232,2.823,8.232,58.649,1.303,-1.414,25.470,-0.654,33.179,-1.665,12.439,163.420
14,100014,b,-999.000,109.412,14.398,17.323,0.472,17.323,62.565,1.774,-0.272,22.552,1.389,40.013,1.856,75.197,198.616
15,100015,s,111.026,32.096,75.271,23.067,3.205,23.067,69.649,1.276,-1.414,30.606,-1.107,39.043,-1.944,19.959,122.176
17,100017,s,127.861,50.953,77.267,26.967,2.833,26.967,79.503,1.586,1.401,30.739,-0.635,48.764,-0.343,17.557,211.720
18,100018,b,-999.000,85.186,68.827,5.042,2.116,5.042,71.443,1.558,-1.351,27.931,1.175,43.512,2.332,44.698,151.816
19,100019,b,-999.000,88.767,115.058,15.337,2.879,15.337,58.211,0.875,-1.395,31.046,1.380,27.165,-1.486,72.981,115.145


In [22]:
feature_tr_grp[0].drop(columns = ['Id','Prediction']).mean(axis=0)

DER_mass_MMC                  -172.077816
DER_mass_transverse_met_lep     58.786239
DER_mass_vis                    81.870309
DER_pt_h                        13.823867
DER_deltar_tau_lep               2.664961
DER_pt_tot                      13.823867
DER_sum_pt                      76.377011
DER_pt_ratio_lep_tau             1.392763
DER_met_phi_centrality          -0.910077
PRI_tau_pt                      34.012723
PRI_tau_eta                     -0.024858
PRI_lep_pt                      42.364287
PRI_lep_eta                     -0.052311
PRI_met                         31.536761
PRI_met_sumet                  125.860810
dtype: float64

In [20]:
x_tr0,mean0,std0 = standardize(feature_tr_grp[0].drop(columns = ['Id','Prediction']).values)
mean0

array([-1.72077816e+02,  5.87862388e+01,  8.18703092e+01,  1.38238670e+01,
        2.66496128e+00,  1.38238669e+01,  7.63770107e+01,  1.39276344e+00,
       -9.10076857e-01,  3.40127231e+01, -2.48576361e-02,  4.23642875e+01,
       -5.23114410e-02,  3.15367606e+01,  1.25860810e+02])

In [23]:
x_tr0.shape

(99913, 15)

In [13]:
# for df['PRI_jet_num'] == 0
x_tr0,_,_ = standardize(feature_tr_grp[0].drop(columns = ['Id','Prediction']).values)
y_tr0 = feature_tr_grp[0].Prediction.replace(['s','b'],[1,-1])
w0 = least_squares(y_tr0,x_tr0)
idx_tr0 = feature_tr_grp[0].Id
y_pred_tr0 = predict_labels(w0, x_tr0)

x_te0,_,_ = standardize(feature_te_grp[0].drop(columns = ['Id','Prediction']).values)
idx_te0 = feature_te_grp[0].Id
y_pred_te0 = predict_labels(w0, x_te0).astype(int)

LinAlgError: Singular matrix

In [316]:
# for df['PRI_jet_num'] == 1
x_tr1,_,_ = standardize(feature_tr_grp[1].drop(columns = ['Id','Prediction']).values)
y_tr1 = feature_tr_grp[1].Prediction.replace(['s','b'],[1,-1])
w1 = least_squares(y_tr1,x_tr1)
idx_tr1 = feature_tr_grp[1].Id
y_pred_tr1 = predict_labels(w1, x_tr1)

x_te1,_,_ = standardize(feature_te_grp[1].drop(columns = ['Id','Prediction']).values)
idx_te1 = feature_te_grp[1].Id
y_pred_te1 = predict_labels(w1, x_te1).astype(int)

In [317]:
# for df['PRI_jet_num'] == 2,3
x_tr2,_,_ = standardize(feature_tr_grp[2].drop(columns = ['Id','Prediction']).values)
y_tr2 = feature_tr_grp[2].Prediction.replace(['s','b'],[1,-1])
w2 = least_squares(y_tr2,x_tr2)
idx_tr2 = feature_tr_grp[2].Id
y_pred_tr2 = predict_labels(w2, x_tr2)

x_te2,_,_ = standardize(feature_te_grp[2].drop(columns = ['Id','Prediction']).values)
idx_te2 = feature_te_grp[2].Id
y_pred_te2 = predict_labels(w2, x_te2).astype(int)

In [318]:
y_pred_te = np.concatenate((y_pred_te0,y_pred_te1,y_pred_te2))
idx_te = np.concatenate((idx_te0,idx_te1,idx_te2))

In [319]:
create_csv_submission(idx_te, y_pred_te, 'submission.csv')

In [320]:
y_pred_te0.shape

(227458,)

In [321]:
idx_te

array([350000, 350002, 350003, ..., 918230, 918233, 918234])

In [322]:
y_pred_tr = np.concatenate((y_pred_tr0,y_pred_tr1,y_pred_tr2))
idx_tr = np.concatenate((idx_tr0,idx_tr1,idx_tr2))
test = pd.DataFrame({'Id':idx_tr,'Prediction':y_pred_tr})

In [323]:
test.sort_values(by='Id',inplace = True)
test.set_index('Id',inplace = True)

In [324]:
y_pred_tr = test.Prediction.get_values().astype(int)

In [325]:
y = data_tr.Prediction.replace(['s','b'],[1,-1]).get_values()

In [326]:
sum(abs(y_pred_tr-y))/len(y)

0.685016